In [1]:
import os
import torch
import albumentations as A
import numpy as np
from albumentations.pytorch import ToTensorV2
import pandas as pd
from torch.utils.data import DataLoader

from Load_Data import ImageDataset, random_seed, get_transforms
from Preprocess import Augmentation
from Train import Trainer
from Inference import Model_Ensemble, tta, run_inference

In [2]:
train_csv_path = "../data/train.csv"
test_csv_path = "../data/sample_submission.csv"

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

random_seed(42) # 시드 고정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 512  # EfficientNetV2 XL 권장 입력 크기
LR = 2e-5  
EPOCHS = 60
BATCH_SIZE = 6
Drop_out = 0.5
weight_decay = 1e-4
num_workers = 4
num_classes = 17
n_splits = 3  # K-fold 
patience = 5  # Early stopp
# tta_iterations = 1  # TTA

model_name = "tf_efficientnetv2_xl_in21ft1k"  
run_name_prefix = "V12"
train_path = "../data/train/"
test_path = "../data/test/"
fold_paths = "../model/V12/" # 학습된 fold 모델 경로
model_save_path = "../model/V12" # 모델 저장 경로
augmented_save_path = "../data/augment_image/V12-augmented" # 증강 이미지 저장 경로
augmented_save_csv_path = "../data/augment_csv/V12-augmented.csv" # 증강 csv 저장 경로
submission_save_path = "../data/submission/V12-submission.csv" # 추론 결과

In [4]:
# Augmentation
augmenter = Augmentation(
    train_df=train_df,
    train_path=train_path, 
    save_path=augmented_save_path,
    img_size=img_size,
    save_csv_path=augmented_save_csv_path 
)
aug_df = augmenter.run(target_count=300) # Class 별 300

# Train
trainer = Trainer(
    df=aug_df,                            # 증강된 데이터 사용
    data_path=augmented_save_path,        # 증강된 이미지 경로 사용
    model_name=model_name,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    drop_out=Drop_out,
    weight_decay=weight_decay,
    img_size=img_size,
    num_workers=num_workers,
    device=device,
    save_dir=model_save_path,
    run_name_prefix=run_name_prefix,
    num_classes=num_classes,
    n_splits=n_splits,
    patience=patience,
    k_fold=True
)

f1_df = trainer.run()
fold_weights = f1_df["f1"].values / f1_df["f1"].sum()

[INFO] Class 0: 현재 100개 → 목표 300개로 증강 중...


[Class 0] 원본 복사: 100%|██████████| 100/100 [00:00<00:00, 254.87it/s]


[INFO] Class 0: 추가 증강 200개 생성 중...


[Class 0] 증강 생성: 100%|██████████| 200/200 [00:08<00:00, 24.02it/s]


[INFO] Class 1: 현재 46개 → 목표 300개로 증강 중...


...

[Fold 2] Ep20 - Train: 0.0795 | Val: 0.6878, Acc: 0.9259, F1: 0.9261


[Fold 2][Epoch 21/60] Training:  11%|█         | 63/567 [00:22<03:00,  2.79it/s, loss=0.00311] 


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f22f742a160>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f24f57b88e0, execution_count=4 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7f25040e72b0, raw_cell="# Augmentation
augmenter = Augmentation(
    train.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a2243565f50726f6a656374227d/root/dev/upstageailab-cv-classification-cv_1/code/Models-Final.ipynb#W3sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import cv2

def get_transforms(img_size):
    train_transform = A.Compose([
        A.LongestMaxSize(max_size=img_size, interpolation=cv2.INTER_AREA),
        # A.Resize(height=img_size, width=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size,
                      border_mode=cv2.BORDER_CONSTANT, fill=(255, 255, 255), p=1),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    val_transform = A.Compose([
        A.LongestMaxSize(max_size=img_size, interpolation=cv2.INTER_AREA),
        # A.Resize(height=img_size, width=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size,
                      border_mode=cv2.BORDER_CONSTANT, fill=(255, 255, 255), p=1),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    return train_transform, val_transform

In [ ]:
# Test Data Loader
test_transform = get_transforms(img_size)[1] 
test_dataset = ImageDataset(test_df, path=test_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# TTA 
tta_transforms = tta()

# Inference
ensembler = Model_Ensemble(
    model_name=model_name,
    fold_paths_dir=fold_paths,  
    device=device,
    num_classes=num_classes,
    drop_out=Drop_out,
    fold_weights=fold_weights,
    k_fold=True 
)

run_inference(
    test_loader=test_loader,
    ensembler=ensembler,
    submission_df=test_df.copy(),
    save_path=submission_save_path,
    tta_transforms=None # TTA X 
)

Inference: 100%|██████████| 524/524 [02:30<00:00,  3.49it/s]

[✓] Saved submission to: ../data/submission/V11-submission.csv
